### to run this code, you need to have standalone EMBOSS (getorf), BLAST, and HMMER installed

In [67]:
import subprocess
import os
from glob import glob
from Bio import SearchIO
from Bio import SeqIO

## load and adjust all paths

In [68]:
outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta")
hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"

### Extract ORFs from genomes

In [69]:
def run_getorf(genome, orf_file):
    command = ["getorf", "-sequence", genome, "-outseq", orf_file, "-table", "0", "-minsize", "360", "-find", "1"] #adjust (120 aa, START to STOP, standard gen. code)
    subprocess.run(command, check=True)

outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta") #adjust - this is just a test dataset

for genome in genomes:
    orf_file = f"{outdir}{genome.split('/')[-1].replace('.fasta', '_orfs.fa')}"
    run_getorf(genome, orf_file)

Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)


KeyboardInterrupt: 

### hmmsearch with HMM against those extracted ORFs

In [ ]:
# running hhmmsearch against multiple databases at once and saving the resulting files using the .hmmout suffix
def run_hmmsearch(orf_file, hmm_model, output_file):
    command = ["hmmsearch", "-o", output_file, hmm_model, orf_file]
    subprocess.run(command, check=True)

hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases


# iterate over the list of database files and run hmmsearch 
for orf_file in orf_files:
    output_file = f"{output_directory}{orf_file.split('/')[-1].replace('_orfs.fa', '_atp.hmmout')}" 
    run_hmmsearch(orf_file, hmm_model, output_file) #needs to be nested inside the loop!


### saving IDs of homologs above inclusion threshold found by HMMER (in all the genomes)

In [ ]:
# saving IDs of sequences above the inclustion treshold into ids_list
ids_list_orfs = []
for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    hmmer_file = SearchIO.read(file, "hmmer3-text")
    for record in hmmer_file:
        if record.is_included:
            ids_list_orfs.append(record.id)
len(ids_list_orfs) 


In [ ]:
print(ids_list_orfs)


### Extracting scaffolds with the viral homologs found by HMMER (ids_list), scaffolds shorter than 18000 nt are discarded

In [70]:
# ids_list contains IDs of ORFs, not scaffolds! this list needs to be modified accordingly
ids_list_scaffolds = []
for id in ids_list_orfs:
    id_modified = id.rsplit('_', 1)[0] # removing the number of orf -> id of the scaffold
    ids_list_scaffolds.append(id_modified)
print(ids_list_scaffolds)

['NODE_5080_length_5696_cov_26.6876', 'NODE_381_length_27838_cov_14.2215', 'NODE_7293_length_4272_cov_22.9291', 'NODE_5397_length_5445_cov_37.4879', 'NODE_6066_length_4954_cov_14.3046', 'NODE_847_length_17049_cov_12.9829', 'NODE_7343_length_4249_cov_22.2454', 'NODE_19673_length_1550_cov_34.6107', 'NODE_1409_length_12728_cov_15.5927', 'NODE_1574_length_11923_cov_14.5027', 'NODE_711_length_18631_cov_14.7806', 'NODE_15445_length_2031_cov_23.9084', 'NODE_15112_length_2083_cov_16.6144', 'NODE_12470_length_2567_cov_17.4088', 'NODE_34064_length_837_cov_18.9655', 'NODE_4057_length_6629_cov_27.157', 'NODE_865_length_16875_cov_12.6059', 'NODE_869_length_16855_cov_12.4798', 'NODE_6477_length_4696_cov_21.0899', 'NODE_17368_length_1781_cov_15.3621', 'NODE_18418_length_1671_cov_21.013', 'NODE_352_length_29307_cov_12.7034', 'NODE_31105_length_924_cov_16.8895', 'NODE_4114_length_6570_cov_18.8181', 'NODE_64132_length_412_cov_37.521', 'NODE_21363_length_1410_cov_17.5911', 'NODE_729_length_18418_cov_17.5

#### to be able to extract the scaffolds, blastable databases need to be created first (for genome and orfs)

In [ ]:
def makeblastdb(file_path, db_type, blastables_outdir):
    db_name = f"{blastables_outdir}/{file_path.split('/')[-1].split('.')[0]}"
    command = ["makeblastdb", "-in", file_path, "-dbtype", db_type, "-out", db_name, "-parse_seqids"]
    subprocess.run(command, check=True)

blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"


for genome_path in genomes:
    makeblastdb(genome_path, "nucl", blastables_outdir) # misto file_path volame s genome_path

for orf_file_path in orf_files:
    makeblastdb(orf_file_path, "prot", blastables_outdir)


In [ ]:
# enables to skip IDs not present in the particular database and proceeds to another one as the old verison of blast does

def run_blastdbcmd(id, genome_db, outscaffold):
    # check if the output file exists and is non-empty
    if os.path.exists(outscaffold) and os.path.getsize(outscaffold) > 0:
        record = SeqIO.read(outscaffold, "fasta")
        if len(record.seq) < 20000:
            print(f"Notice: Scaffold {id} is shorter than 20000 nucleotides, removing file.")
            os.remove(outscaffold)
            return False
        return True

    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold]
    result = subprocess.run(command, text=True, capture_output=True)

    if result.returncode == 0:
        # check the file size right after running the command
        if os.path.getsize(outscaffold) > 0:
            record = SeqIO.read(outscaffold, "fasta")
            if len(record.seq) < 20000:
                print(f"Notice: Scaffold {id} is shorter than 20000 nucleotides, removing file.")
                os.remove(outscaffold)
                return False
            return True
        else:
            print(f"Notice: The file created for ID {id} is empty, removing.")
            os.remove(outscaffold)
            return False
    else:
        print(f"Warning: ID {id} not found in database {genome_db}.")
        if os.path.exists(outscaffold) and os.path.getsize(outscaffold) == 0:
            os.remove(outscaffold)
        return False

scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
prot_database = "_orfs." # to exclude orf files; we need genomes

for id in ids_list_scaffolds:
    for filename in os.listdir(blastables_outdir):
        if prot_database not in filename:
            base_name = filename.split('.')[0]
            genome_db = os.path.join(blastables_outdir, base_name)
            outscaffold = f"{scaffolds_dir}/{id}_EVE.fasta"
            if run_blastdbcmd(id, genome_db, outscaffold):
                break  # Exit the loop if successful


## Turning the extracted scaffolds into single line files to make it possible to search for the repeats flanking the EVEs

In [ ]:
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
singleline_dir = "/home/majnusova/all/projects/plv/data/eve_singleline_scaffolds/" 

for outscaffold in os.listdir(scaffolds_dir):  #os.listdir lists all files/directories; not paths
    outscaffold_path = os.path.join(scaffolds_dir, outscaffold) # paths to all the files listed
    with open(outscaffold_path, "r") as infile:
        lines = infile.readlines()
    header = lines[0]  # fasta header
    joined_lines = ''.join(line.strip() for line in lines[1:])  # Join all lines except the first

    output_file_path = os.path.join(singleline_dir, outscaffold)  
    with open(output_file_path, "w") as outfile:
        outfile.write(header) 
        outfile.write(joined_lines)  

## blastn to find DRs (shortest one has 48 nt, longest one 101 nt) - je treba vybrat jen cast skafoldu, nektere jsou velke?

In [89]:
ids_dict_orfs = {} #dictionary of lists - potrebujeme vedet, proti jakemu genomu scaffoldy blastovat, vracime se zpet k hemmru (IDs casto neobsahuji nazev genomu, ze ktereho pochazi)

for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    hmm_records = SearchIO.read(file, "hmmer3-text")
    ids_dict_orfs[file] = [] # create empty list in the dictionary for the current file
    for record in hmm_records:
        if record.is_included:
            ids_dict_orfs[file].append(record.id) # append the ID to the list associated with the current file

""" 
# count the total number of IDs across all files:
total_ids = sum(len(id_list) for id_list in ids_dict_orfs.values())
print(f"Total number of IDs across all files: {total_ids}")

# To see the count of IDs per file:
for file, id_list in ids_dict_orfs.items():
    print(f"{file}: {len(id_list)} IDs")
    """


' \n# count the total number of IDs across all files:\ntotal_ids = sum(len(id_list) for id_list in ids_dict_orfs.values())\nprint(f"Total number of IDs across all files: {total_ids}")\n\n# To see the count of IDs per file:\nfor file, id_list in ids_dict_orfs.items():\n    print(f"{file}: {len(id_list)} IDs")\n    '

In [90]:
print(ids_dict_orfs)

{'/home/majnusova/all/projects/plv/data/eustigs_hmmout/Chic10_scaffolds_atp.hmmout': ['NODE_5080_length_5696_cov_26.6876_1', 'NODE_381_length_27838_cov_14.2215_25', 'NODE_7293_length_4272_cov_22.9291_3', 'NODE_5397_length_5445_cov_37.4879_2', 'NODE_6066_length_4954_cov_14.3046_4', 'NODE_847_length_17049_cov_12.9829_3', 'NODE_7343_length_4249_cov_22.2454_2', 'NODE_19673_length_1550_cov_34.6107_1', 'NODE_1409_length_12728_cov_15.5927_2', 'NODE_1574_length_11923_cov_14.5027_1', 'NODE_711_length_18631_cov_14.7806_4', 'NODE_15445_length_2031_cov_23.9084_2', 'NODE_15112_length_2083_cov_16.6144_1', 'NODE_12470_length_2567_cov_17.4088_2', 'NODE_34064_length_837_cov_18.9655_1', 'NODE_4057_length_6629_cov_27.157_1', 'NODE_865_length_16875_cov_12.6059_4', 'NODE_869_length_16855_cov_12.4798_4', 'NODE_6477_length_4696_cov_21.0899_1', 'NODE_17368_length_1781_cov_15.3621_1', 'NODE_18418_length_1671_cov_21.013_1', 'NODE_352_length_29307_cov_12.7034_4', 'NODE_31105_length_924_cov_16.8895_1', 'NODE_4114

In [ ]:
def blastn_repeats(input, output, database):
    command = ["blastn", "-query", input, "-out", output, "-db", db] 
    subprocess.run(command, check=True)


output_dir = "/home/majnusova/all/projects/plv/data/blastn_repeats/" # vytvorit na kazdy genom slozku zvlast!
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/" # contains the assemblies

for file in os.listdir(singleline_dir):
    input = os.path.join(singleline_dir, file)
    output = os.path.join(output_dir, file)
    database = 
    
    

blastn({input}, {output}, {database})

## TSD - nejspis nelze zautomatizovat? nebo zkusit vyextrahovat malou cast scaffoldu, treba plus 10 nt down i upstream od oubou casti repetice a projet to blastn-short?

## extract orfs between tirs - pujde v pohode i bez vymezeneho TSD. kontrola spravnosti elementu - v orfech musi byt protein identifikovany hmmerem hned na zacatku... 